In [2]:
#import numpy as np
#import pandas as pd

In [3]:
Fin_data = pd.read_excel('./Fin_data_01.xlsx')
Fin_data

,Year,Month - name,Month -sequence,Date,Business Line,"Amount, $",Expense subgroup,Revenue / Expense Group,Revenue or expense
0,2023,January,1,2023-01-31,Nutrition and Food Supplements,153000,NaN,Sales,Revenue
1,2023,January,1,2023-01-31,Nutrition and Food Supplements,27000,NaN,Consulting and professional services,Revenue
2,2023,January,1,2023-01-31,Nutrition and Food Supplements,6000,NaN,Other income,Revenue
3,2023,January,1,2023-01-31,Nutrition and Food Supplements,-15000,Rent,Opex,Expense
4,2023,January,1,2023-01-31,Nutrition and Food Supplements,-9000,Equipment,Opex,Expense
...,...,...,...,...,...,...,...,...,...
575,2023,December,1,2023-12-31,Sportswear,-4000,Packaging,COGS,Expense
576,2023,December,1,2023-12-31,Sportswear,-7000,Shipping,COGS,Expense
577,2023,December,1,2023-12-31,Sportswear,-9000,Sales,COGS,Expense
578,2023,December,1,2023-12-31,Sportswear,-110000,Labor,COGS,Expense


In [4]:
Fin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     580 non-null    int64         
 1   Month - name             580 non-null    object        
 2   Month -sequence          580 non-null    int64         
 3   Date                     580 non-null    datetime64[ns]
 4   Business Line            580 non-null    object        
 5   Amount, $                580 non-null    int64         
 6   Expense subgroup         468 non-null    object        
 7   Revenue / Expense Group  580 non-null    object        
 8   Revenue or expense       580 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 40.9+ KB


In [5]:
Fin_data.describe()

,Year,Month -sequence,Date,"Amount, $"
count,580.0,580.0,580,580.000000
mean,2023.0,1.0,2023-07-15 01:49:14.482758656,7438.137931
min,2023.0,1.0,2023-01-31 00:00:00,-234000.000000
25%,2023.0,1.0,2023-03-31 00:00:00,-30000.000000
50%,2023.0,1.0,2023-06-30 00:00:00,-10800.000000
75%,2023.0,1.0,2023-09-30 00:00:00,-1800.000000
max,2023.0,1.0,2023-12-31 00:00:00,750000.000000
std,0.0,0.0,NaN,118699.587071


In [6]:
Fin_data.columns

Index(['Year', 'Month - name', 'Month -sequence', 'Date', 'Business Line',
       'Amount, $', 'Expense subgroup', 'Revenue / Expense Group',
       'Revenue or expense'],
      dtype='object')

In [9]:
#Now we shall rename columns

Fin_data.rename(columns= {'Month - name': 'Month_name'}, inplace= True)

In [10]:
Fin_data.rename(columns= {'Month -sequence' : 'Month_sequence', 'Amount, $': 'Amount', 'Expense subgroup' : 'Expense_subgroup', 'Revenue or expense' : 'Rev_or_exp'
                          , 'Revenue / Expense Group' : 'Rev_exp_group','Business Line': 'Biz_line' }, inplace= True)

In [11]:
Fin_data

,Year,Month_name,Month_sequence,Date,Biz_line,Amount,Expense_subgroup,Rev_exp_group,Rev_or_exp
0,2023,January,1,2023-01-31,Nutrition and Food Supplements,153000,NaN,Sales,Revenue
1,2023,January,1,2023-01-31,Nutrition and Food Supplements,27000,NaN,Consulting and professional services,Revenue
2,2023,January,1,2023-01-31,Nutrition and Food Supplements,6000,NaN,Other income,Revenue
3,2023,January,1,2023-01-31,Nutrition and Food Supplements,-15000,Rent,Opex,Expense
4,2023,January,1,2023-01-31,Nutrition and Food Supplements,-9000,Equipment,Opex,Expense
...,...,...,...,...,...,...,...,...,...
575,2023,December,1,2023-12-31,Sportswear,-4000,Packaging,COGS,Expense
576,2023,December,1,2023-12-31,Sportswear,-7000,Shipping,COGS,Expense
577,2023,December,1,2023-12-31,Sportswear,-9000,Sales,COGS,Expense
578,2023,December,1,2023-12-31,Sportswear,-110000,Labor,COGS,Expense
